In [13]:
import pandas as pd
import re
import json
import glob
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Define range for file, friends change this help me download when you're free lel
start_int = 300
end_int = 400

In [14]:
# Combine CSV, generate output

files = glob.glob("Clean_data/Kickstarter.csv")

csv = []

for file in files:
    df = pd.read_csv(file, index_col=None, header=0)
    csv.append(df)

df = pd.concat(csv, axis=0, ignore_index=True)

In [15]:
print(df.shape, "\n",df.columns)

(3429, 27) 
 Index(['Unnamed: 0', 'backers_count', 'blurb', 'category',
       'converted_pledged_amount', 'country', 'created_at', 'current_currency',
       'deadline', 'fx_rate', 'goal', 'id', 'launched_at', 'location', 'name',
       'pledged', 'profile', 'slug', 'source_url', 'spotlight', 'staff_pick',
       'state', 'state_changed_at', 'static_usd_rate', 'urls', 'usd_pledged',
       'usd_type'],
      dtype='object')


In [16]:
urls = df.urls
ids = df.id

urls_ = list(map(lambda url:json.loads(url)["web"]["project"],urls))
urls = list(map(lambda url:url.replace("?ref=discovery_category_newest","/comments"),urls_))
#urls = urls_

#get url
#url = df[df.state == "successful"].urls.reset_index().urls[1]
#url = json.loads(url)
#url = url["web"]["project"]
#url

In [ ]:
#path = "Webdriver/chromedriver.exe"
#driver = webdriver.Chrome(path)

from selenium.webdriver.support import ui
from selenium.common.exceptions import TimeoutException

from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome(ChromeDriverManager().install())
driver = webdriver.Chrome('/Users/amandatan/Downloads/chromedriver') 


comments = []
n_comments = []

for i in tqdm(range(start_int,end_int)):
    driver.get(urls[i])
#driver.get(url)

#wait = ui.WebDriverWait(driver, 10)
#wait.until(lambda driver: driver.find_element_by_xpath('//a[@id="comments-emoji"]')).click()
#driver.find_element_by_id('comments-emoji').click()

    # 1. Get total number of updates
    numComments = WebDriverWait(driver,10).until(EC.visibility_of_all_elements_located((By.XPATH,'//*[@id="comments-emoji"]/span')))
    
    for number in numComments:
        n_comments.append(number.text)
        
    # 2. Get all comments and save as a list
    comments_raw = WebDriverWait(driver,10).until(
        EC.presence_of_element_located((By.XPATH,"//div[@id='react-project-comments']"))
    )
    #for comment in comments_raw:

    while True:
        if len(comments_raw.text) >0:
            comments.append(comments_raw.text)
            break


driver.quit()

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [/Users/amandatan/.wdm/drivers/chromedriver/mac64/89.0.4389.23/chromedriver] found in cache
 30%|███       | 30/100 [01:14<02:50,  2.43s/it]

In [6]:
comments

["Only backers can post comments. Log in\nBambi Lewis\nover 7 years ago\nBeautiful snowflakes, Lysette! I've never seen anything like them and recall seeing several hanging in our appartment, almost like a mobil......I can't wait to get mine. Will definitely buy more!",
 "Only backers can post comments. Log in\nRichard Farrow\nover 1 year ago\nWell done, can't wait to see the finished work.",
 'Only backers can post comments. Log in\nNo comments yet.',
 'Only backers can post comments. Log in\nNo comments yet.',
 'Only backers can post comments. Log in\nNo comments yet.',
 'Only backers can post comments. Log in\nNo comments yet.',
 "Only backers can post comments. Log in\nParker Knowles\nalmost 6 years ago\nA project so ambitious and passionate is not easy to finance. This documentary absolutely fills both of those categories. Give the trailer a watch - or three - and get a peek at just how beautiful this film is in both essence and execution. If it moved you, please fund my friends; 

In [7]:
df_export = pd.DataFrame({"id":ids.iloc[start_int:end_int],
                         "comments":comments,
                         "n_comments":n_comments}, index = None)
filename = "Comments_" + str(start_int) + "-" + str(end_int)
df_export.to_csv("/Users/amandatan/Desktop/BT4222_GrpProject/Output/Comments/{}.csv".format(filename))